In [8]:
import time
from os.path import join

!pip install pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import recommender_system

You should consider upgrading via the '/home/rom42pla/anaconda3/bin/python -m pip install --upgrade pip' command.


In [9]:
starting_time = time.time()

# PySpark setup

In [10]:
# creates session and context
conf = SparkConf().set("spark.ui.port", "4050")
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

# Data retrieval

In [11]:
# reads the dataframe
df = spark.read.options(header='True', inferSchema='True', delimiter=',').csv(join(".", "dataset_sample.csv"))
df = recommender_system.preprocess_dataframe(df=df)
df.cache()
df.printSchema()
print(f"|ratings| = {df.count()}, \t"
      f"|users| = {df.select('user_id').distinct().count()}, \t"
      f"|items| = {df.select('category_id').distinct().count()}")
df.show(4)

root
 |-- event_time: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- rating: integer (nullable = false)

|ratings| = 408042, 	|users| = 309931, 	|items| = 596
+--------------------+---------+----------+-----------+------+
|          event_time|  user_id|event_type|category_id|rating|
+--------------------+---------+----------+-----------+------+
|2019-10-01 00:01:...|514218020|      view|-1769995873|     1|
|2019-10-01 00:06:...|533940457|      view|-1769995873|     1|
|2019-10-01 00:07:...|547290577|      view|-1090518581|     1|
|2019-10-01 00:07:...|547949682|      view|-2080373895|     1|
+--------------------+---------+----------+-----------+------+
only showing top 4 rows



# Training

In [12]:
# training the model
training, test = df.randomSplit([0.8, 0.2])
model = recommender_system.train_recommender_system(df=training, iterations=10, logs=True)
# model.save("model")

Starting training of ALS model
Training time: 7s


# Results

In [13]:
total_time = time.time() - starting_time
print(f"Total time: {int(total_time)}s")

Total time: 17s
